In [41]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import csv
import requests
from scipy import stats
headers = { 
    'accept': "application/json", 
    'apikey': "8a6c3ab67caa3a66c274d4455dc0ebc3" } 
cities_df = pd.read_csv("./resources/ListOfCities.csv")
cities_df.head()

,State,City
0,Texas,Dallas
1,California,San Francisco
2,Mississippi,Jackson


In [42]:
base_url ="https://search.onboard-apis.com/areaapi/v2.0.0/state/lookup"
res = requests.get(base_url,headers=headers).json()
res
st_data =pd.DataFrame(res['response']['result']['package']['item'])
st_data.rename(columns={'name':'State','geo_key':'St_geo_key'}, inplace=True)
statedata = st_data[['State','St_geo_key','abbreviation']]
statedata.head()

,State,St_geo_key,abbreviation
0,Alabama,ST01,AL
1,Alaska,ST02,AK
2,Arizona,ST04,AZ
3,Arkansas,ST05,AR
4,California,ST06,CA


In [43]:
city_table =pd.merge(cities_df, statedata, on="State", how="left")
city_table.head()

,State,City,St_geo_key,abbreviation
0,Texas,Dallas,ST48,TX
1,California,San Francisco,ST06,CA
2,Mississippi,Jackson,ST28,MS


In [44]:
appended_new = pd.DataFrame([])
for index, row in city_table.iterrows():
    stgc = row['St_geo_key']
    city = row['City']  
    state = row['State']
    state_abbre = row['abbreviation']
    city_url ="https://search.onboard-apis.com/areaapi/v2.0.0/county/lookup/?StateId="+stgc+"&GeoType=CO" 
    
    resc = requests.get(city_url,headers=headers).json()
    city_data = pd.DataFrame(resc['response']['result']['package']['item'])
    new = city_data.loc[city_data['name'] == city]
    new['state'] = state
    new['state abbreviation'] = state_abbre
    appended_new = appended_new.append(new)
    
appended_new

C:\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
C:\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,geo_center_latitude,geo_center_longitude,geo_key,id,name,type,state,state abbreviation
56,32.7669870000,-96.7784240000,CO48113,CO48113,Dallas,CO,Texas,TX
37,37.7272390000,-123.0322290000,CO06075,CO06075,San Francisco,CO,California,CA
29,30.4560050000,-88.6251010000,CO28059,CO28059,Jackson,CO,Mississippi,MS


In [45]:
appended_zip = pd.DataFrame([])
for index, row in appended_new.iterrows():
    geoid = row['geo_key']
    city = row['name']
    state = row['state']
    state_abbre = row['state abbreviation']
    zip_url ="https://search.onboard-apis.com/areaapi/v2.0.0/geoid/lookup/?geoId="+geoid+"&GeoType=ZI" 
    
    resc_zip = requests.get(zip_url,headers=headers).json()
    city_data = pd.DataFrame(resc_zip['response']['result']['package']['item'])
    city_data['county name']=city
    city_data['state'] = state
    city_data['state abbreviation'] = state_abbre
    appended_zip = appended_zip.append(city_data)
    
appended_zip.head()

,area,area_unit,geo_key,id,name,type,county name,state,state abbreviation
0,0.0052,sq. mi.,ZI75390,75390,Dallas,ZI,Dallas,Texas,TX
1,0.0057,sq. mi.,ZI75270,75270,Dallas,ZI,Dallas,Texas,TX
2,0.4437,sq. mi.,ZI75251,75251,Dallas,ZI,Dallas,Texas,TX
3,0.4525,sq. mi.,ZI75246,75246,Dallas,ZI,Dallas,Texas,TX
4,0.7034,sq. mi.,ZI75202,75202,Dallas,ZI,Dallas,Texas,TX


In [83]:
appended_main = pd.DataFrame([])

for index, row in appended_zip.iterrows():
    cgc = row['geo_key']
    city = row['name']
    county_city = row['county name']
    state = row['state']
    state_abbre = row['state abbreviation']
    
    print(cgc)
  
    base_url="https://search.onboard-apis.com/communityapi/v2.0.0/area/full?AreaId="+cgc
    try:
        response = requests.get(base_url,headers=headers)
        if response.status_code == requests.codes.ok:
            resct = response.json()
            zip_toadd = pd.DataFrame(resct['response']['result']['package']['item'])
            zip_toadd['name']=city
            zip_toadd['county name']=county_city
            zip_toadd['state'] = state
            zip_toadd['state abbreviation'] = state_abbre
            zip_toadd['geo_key'] = cgc
            appended_main = appended_main.append(zip_toadd)
        else:
            print("zip data not found. Skipping...")
    except:         
        print("Unknown exception...")
        pass
        
appended_main.head()

ZI75390
ZI75270
zip data not found. Skipping...
ZI75251
ZI75246
ZI75202
ZI75226
ZI75201
ZI75219
ZI75210
ZI75204
ZI75254
ZI75223
ZI75233
ZI76155
ZI75209
ZI75240
ZI75001
ZI75207
ZI75206
ZI75205
ZI75244
ZI75249
ZI75225
ZI75203
ZI75252
ZI75116
ZI75039
ZI75287
ZI75231
ZI75224
ZI75208
ZI75137
ZI75054
ZI75247
ZI75238
ZI75237
ZI75235
ZI75218
ZI75230
ZI75248
ZI75042
ZI75214
ZI75180
ZI75082
ZI75041
ZI75232
ZI75215
ZI75243
ZI75081
ZI75134
ZI75212
ZI75048
ZI75234
ZI75044
ZI75062
ZI75227
ZI75228
ZI75080
ZI75089
ZI75061
ZI75220
ZI75172
ZI75007
ZI75088
ZI75141
ZI75229
ZI75150
ZI75236
ZI75067
ZI75038
ZI75060
ZI75253
ZI75051
ZI75216
ZI75063
ZI75040
ZI75149
ZI75006
ZI75181
ZI75019
ZI75211
ZI75043
ZI75182
ZI75115
ZI75050
ZI75241
ZI75052
ZI75217
ZI75146
ZI75104
ZI75098
ZI76051
ZI75154
ZI75159
ZI75125
ZI94104
ZI94108
ZI94105
ZI94111
ZI94102
ZI94130
ZI94133
ZI94123
ZI94115
ZI94158
ZI94109
ZI94103
ZI94114
ZI94117
ZI94127
ZI94107
ZI94118
ZI94131
ZI94129
ZI94110
ZI94122
ZI94134
ZI94116
ZI94121
ZI94112
ZI94132


,age00_04,age05_09,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,...,trwother,trwpublic,trwself,two_bed_county,two_bedindex,vph1,vphgt1,vphnone,county name,state abbreviation
0,1,1,0,1,3,3,3,2,2,2,...,0,0,0,1031,0,55,37,8,Dallas,TX
0,376,114,88,23,40,269,463,318,164,101,...,5,8,24,1031,129,40,36,23,Dallas,TX
0,260,187,156,135,147,219,279,280,223,173,...,17,106,32,1031,81,52,33,15,Dallas,TX
0,426,102,76,22,47,239,557,470,283,200,...,13,32,224,1031,136,55,37,9,Dallas,TX
0,447,259,192,129,200,438,552,456,367,306,...,5,170,145,1031,103,62,31,7,Dallas,TX


In [86]:
# geo_key	Full geographic identifier used for searching.
# latitude	Centroid latitude for the area.    
# longitude	Centroid longitude for the area.	
# airportdist	Distance to closest airport with more than 10,000 boardings annually. Calculated from the geographic centroid of the Census Zip Area (CZA).
# airx	A score that represents the relative air quality by combining indices of Ozone, Carbon Monoxide, Lead, Nitrogen Oxide, and Particulate matter and comparing to the national average of 100. A score of 200 indicates twice the presence of air pollutants than the national average, while 50 indicates half the presence. The different types of pollution are given equal weight in this score.	
# ancestamer	Number of people living within the area of American descent.
# ancestamind	Number of people living within the area of Native American descent.
# ancestasian	Number of people living within the area of Asian descent.
# ancesteuro	Number of people living within the area of European descent.
# ancesthawai	Number of people living within the area of Hawaiian descent.
# ancesthisp	Number of people living within the area of Hispanic descent.
# ancestother	Number of people living within the area of a descent listed as Other.
# ancestunclassified	Number of people living within the area of an unclassified descent.
# avgsaleprice	Average property tax amount for the area.
# avg_prop_tax_year_count	Average residential home sales price for four recent quarters. Based on actual sales collected from Assessor's offices. All included sales are arms length transactions greater than $25,000 representing single family and multi-family (2 - 4) residential sales.  * Note: This field is only available for approximately 2,500 counties nationally.
# cocrmcytotc	Only included for ZI, PL, CS and TR level records: A score that represents the combined risks of rape, murder, assault, robbery, burglary, larceny and vehicle theft compared to the national average of 100. A score of 200 indicates twice the national average total crime risk, while 50 indicates half the national risk. The different types of crime are given equal weight in this score.
# crmcytotc	A score that represents the combined risks of rape, murder, assault, robbery, burglary, larceny and vehicle theft compared to the national average of 100. A score of 200 indicates twice the national average total crime risk, while 50 indicates half the national risk. The different types of crime are given equal weight in this score.
# daypop	The daytime population in this area.
# edubach	Number of people who achieved a Bachelor's Degree.
# eduagelt9_00	Education attainment from Census 2000: Number of people who achieved an 8th grade education.
# eduhsch	Number of people who are high school graduates.
# expfoodbev	The costs associated with food at home, food away from home; restaurants, catered affairs, non-alcoholic and alcoholic beverages. Index score (100=National Average)
# exphealth	The costs associated with health care including: health insurance, medicare payments, medical services, prescription and non prescription drugs and medical supplies. Index score (100=National Average)
# exphh	The costs associated with owned dwellings; maintenance, repairs, homeowners insurance, property management and security. Index score (100=National Average)
# exphhops	The costs associated with household operations; in home babysitting and child care, day care, nursery and preschools. Also includes other household expenses like: housekeeping services and supplies, lawn care services, home security, pest control, appliance repair and rental as well as other household products and postage. Index score (100=National Average)
# exptotal	Average annual expenditures for food, beverages, housing, utilities, household operations, household supplies and furnishings, apparel, transportation, health care, entertainment, personal care, personal insurance, education and miscellaneous. (100=National Average)
# exptransport	The costs associated with new and used vehicle purchases, vehicle finance charges, gasoline and oil, vehicle repair and maintenance, vehicle insurance, parking and tolls, public transportation and other transportation costs. Index score (100=National Average)
# three_bedindex	Localized index to show expected local pricing variations for a three bedroom dwelling.
# three_bed_county	Gross rent estimate for a three bedroom apartment; includes the shelter rent plus the cost of all tenant-paid utilities, except telephones, cable or satellite television service, and internet service.
# idxexptotal	The average of the total expenses shown within an area per household as it relates to the national average. Index score (100=National Average)
# inccypcap	Average individual income of people in the area.
# inccyavehh	Average income of households in the area.
# medsaleprice	Median residential home sales price for four recent quarters. Based on actual sales collected from Assessor's offices. All included sales are arms length transactions greater than $25,000 representing single family and multi-family (2 - 4) residential sales. Note: This field is only available for approximately 2,500 counties nationally.
# ozone	An invisible gas that irritates and impairs breathing. Index score (100=National Average).
#  pm10	Fine particles of pollutants which tend to reduce visibility and invade the lungs deeply. Index score (100=National Average)
# precipann	Combined average annual rainfall and snowfall in inches.
# rskcyhanx	The total risk of damaging hailstorms. Based on analysis of historical hail frequency and severity. Index score (100=National Average)
# rskcyhunx	The total hurricane risk. Based on analysis of historical hurricane tracks and intensity. Index score (100=National Average)
# rskcyquak	The total earthquake risk. Based on analysis of historical earthquake magnitudes and fault locations. Index score (100=National Average)
# rskcyrisk	The total risk from various storm types (hurricanes, tornadoes, damaging hailstorms and damaging wind storms). Index score (100=National Average)
# salestaxrate	The reported sales tax rate for the area. Additional local taxes may apply (see SALESTAXTYPE).
# trwave	The average commute time of workers living in this area.


raw_all_df = appended_main[['geo_key','name','county name','state','state abbreviation',\
                            'longitude','latitude',\
                            'airportdist','airx','ancestamer',\
                            'ancestamind','ancestasian','ancesteuro',\
                            'ancesthawai','ancesthisp','ancestother',\
                            'ancestunclassified','avgsaleprice',\
                           'avg_prop_tax_year_count','cocrmcytotc',\
                           'crmcytotc','daypop','edubach','eduagelt9_00',\
                           'eduhsch','expfoodbev','exphealth',\
                           'exphh','exphhops','exptotal','exptransport',\
                           'three_bedindex','three_bed_county','idxexptotal',\
                           'inccypcap','inccyavehh','medsaleprice',\
                           'ozone','pm10','precipann','rskcyhanx',\
                           'rskcyhunx','rskcyquak','rskcyrisk','salestaxrate',\
                           'trwave']]
raw_all_df

,geo_key,name,county name,state,state abbreviation,longitude,latitude,airportdist,airx,ancestamer,...,medsaleprice,ozone,pm10,precipann,rskcyhanx,rskcyhunx,rskcyquak,rskcyrisk,salestaxrate,trwave
0,ZI75390,Dallas,Dallas,Texas,TX,-96.840650,32.814164,2,0,0,...,,0,0,37.05,0,0,0,0,6.25,53
0,ZI75251,Dallas,Dallas,Texas,TX,-96.771521,32.917204,7,0,110,...,221151,0,0,40.39,0,0,0,0,6.25,16
0,ZI75246,Dallas,Dallas,Texas,TX,-96.770643,32.795407,6,0,27,...,267187,0,0,37.05,0,0,0,0,6.25,30
0,ZI75202,Dallas,Dallas,Texas,TX,-96.803858,32.781909,5,0,99,...,201250,0,0,37.05,0,0,0,0,6.25,19
0,ZI75226,Dallas,Dallas,Texas,TX,-96.774934,32.786490,6,0,65,...,142500,0,0,37.05,0,0,0,0,6.25,28
0,ZI75201,Dallas,Dallas,Texas,TX,-96.801727,32.788921,5,0,407,...,505875,0,0,37.05,0,0,0,0,6.25,16
0,ZI75219,Dallas,Dallas,Texas,TX,-96.813994,32.811967,3,0,466,...,268850,0,0,37.05,0,0,0,0,6.25,19
0,ZI75210,Dallas,Dallas,Texas,TX,-96.742837,32.768991,8,0,80,...,38265,0,0,37.05,0,0,0,0,6.25,32
0,ZI75204,Dallas,Dallas,Texas,TX,-96.790236,32.803217,5,0,1161,...,352837,0,0,37.05,0,0,0,0,6.25,21
0,ZI75254,Dallas,Dallas,Texas,TX,-96.807231,32.943798,7,0,450,...,157500,0,0,40.39,0,0,0,0,6.25,22


In [87]:
raw_all_df.shape

(131, 46)

In [88]:
raw_all_df1 = raw_all_df.reset_index()
raw_all_reset = raw_all_df1.drop(columns=['index'])

raw_all_reset

,geo_key,name,county name,state,state abbreviation,longitude,latitude,airportdist,airx,ancestamer,...,medsaleprice,ozone,pm10,precipann,rskcyhanx,rskcyhunx,rskcyquak,rskcyrisk,salestaxrate,trwave
0,ZI75390,Dallas,Dallas,Texas,TX,-96.840650,32.814164,2,0,0,...,,0,0,37.05,0,0,0,0,6.25,53
1,ZI75251,Dallas,Dallas,Texas,TX,-96.771521,32.917204,7,0,110,...,221151,0,0,40.39,0,0,0,0,6.25,16
2,ZI75246,Dallas,Dallas,Texas,TX,-96.770643,32.795407,6,0,27,...,267187,0,0,37.05,0,0,0,0,6.25,30
3,ZI75202,Dallas,Dallas,Texas,TX,-96.803858,32.781909,5,0,99,...,201250,0,0,37.05,0,0,0,0,6.25,19
4,ZI75226,Dallas,Dallas,Texas,TX,-96.774934,32.786490,6,0,65,...,142500,0,0,37.05,0,0,0,0,6.25,28
5,ZI75201,Dallas,Dallas,Texas,TX,-96.801727,32.788921,5,0,407,...,505875,0,0,37.05,0,0,0,0,6.25,16
6,ZI75219,Dallas,Dallas,Texas,TX,-96.813994,32.811967,3,0,466,...,268850,0,0,37.05,0,0,0,0,6.25,19
7,ZI75210,Dallas,Dallas,Texas,TX,-96.742837,32.768991,8,0,80,...,38265,0,0,37.05,0,0,0,0,6.25,32
8,ZI75204,Dallas,Dallas,Texas,TX,-96.790236,32.803217,5,0,1161,...,352837,0,0,37.05,0,0,0,0,6.25,21
9,ZI75254,Dallas,Dallas,Texas,TX,-96.807231,32.943798,7,0,450,...,157500,0,0,40.39,0,0,0,0,6.25,22


In [89]:
#output the data set for analysis
outputfile = "Full Data For Analysis.csv"
raw_all_reset.to_csv(outputfile)